In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [0]:
%pip install openpyxl

In [0]:
dbutils.library.restartPython()

In [0]:
file_path = "/dbfs/FileStore/shared_uploads/revathy.s@diggibyte.com/tmp/sample_data_one_translated-1.xlsx"

df = pd.read_excel(file_path, header=[0, 1]) 

# Flatten column headers 
df.columns = ['_'.join(map(str, col)).strip('_') for col in df.columns]

df.rename(columns={df.columns[0]: "Model"}, inplace=True)

df = df.dropna(how='all')

for col in df.columns:
    if col != "Model":
        df[col] = pd.to_numeric(df[col], errors='coerce')

display(df)


In [0]:
spark_df = spark.createDataFrame(df)
spark_df.printSchema()

In [0]:
df_selected =spark_df.drop("Total turnover_TOTAL", "4627500_unit price", "4627500_The total amount", "4627500_Inventory at the end of the month (Company-wide)","Model_Unnamed: 18_level_1")

df_filtered = df_selected.filter(col("Model").isNotNull())

display(df_filtered)

In [0]:
%python
unpivoted = df_filtered.selectExpr(
    "Model",
    "stack(13, " +
    "'Shinko Mitsukoshi_Taichung', `Shinko Mitsukoshi_Taichung`, " +
    "'Shinko Mitsukoshi_North Station', `Shinko Mitsukoshi_North Station`, " +
    "'Shinko Mitsukoshi_Peach Station', `Shinko Mitsukoshi_Peach Station`, " +
    "'Shinko Mitsukoshi_Chiayi', `Shinko Mitsukoshi_Chiayi`, " +
    "'Shinko Mitsukoshi_Simon', `Shinko Mitsukoshi_Simon`, " +
    "'Shinko Mitsukoshi_Left Battalion', `Shinko Mitsukoshi_Left Battalion`, " +
    "'Shinko Mitsukoshi_A8', `Shinko Mitsukoshi_A8`, " +
    "'Shinko Mitsukoshi_Heavenly Mother', `Shinko Mitsukoshi_Heavenly Mother`, " +
    "'SOGO_Revival', `SOGO_Revival`, " +
    "'SOGO_Hsinchu', `SOGO_Hsinchu`, " +
    "'Taipei_Three creations', `Taipei_Three creations`, " +
    "'outlet_Tainan', `outlet_Tainan`, " +
    "'outlet_Kaohsiung', `outlet_Kaohsiung`" +
    ") as (store_name, sales_qty)"
)

final_df = unpivoted.withColumn("Country", lit("Taiwan"))\
    .withColumn("Key_account", lit("Gaston"))\
        .withColumn("load_date", current_date())\
            .withColumnRenamed("Model", "Product_code")

# display(final_df)

In [0]:

final_df = final_df.withColumn("load_date", col("load_date").cast("date"))\
    .withColumn("year", split(col("load_date").cast("string"), "-").getItem(0))\
    .withColumn("month", split(col("load_date").cast("string"), "-").getItem(1))\
    .withColumn("day", split(col("load_date").cast("string"), "-").getItem(2))\
    .drop("load_date")

df = final_df.filter(col("sales_qty").isNotNull())

# display(df)